In [2]:
from tqdm import tqdm
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn


conn = get_db_connection()
cursor = conn.cursor()
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
import matplotlib.pyplot as plt
import warnings
import numpy as np
cursor.execute(f"SELECT distinct code from public.maincode where listed = 'us100';")
conn.commit()
list_ = [i[0] for i in cursor.fetchall()]

cursor.execute(f"SELECT distinct code from public.maincode where listed = 'us500';")
conn.commit()
list_500 = [i[0] for i in cursor.fetchall()]

In [ ]:
# us500 thetadata
import requests
import threading
import time
from datetime import datetime, timedelta
import psycopg2
from IPython.display import display, clear_output
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
class ContinuousProcessor:
    def __init__(self):
        self.market_open = True
        self.heads_code = None

    def lineNotify(self, code, direction, stdized123):
        url = 'https://notify-api.line.me/api/notify'
        line_token = '4YeT18qmmjfGAM40QTmlZqNicCf36sweYK5CAMdEmRS'
        headers = {
            'Authorization': 'Bearer ' + line_token    # 設定權杖
        }
        data = {
            'message':f'{code}\n{direction}\n{datetime.now()}\n{stdized123}'     # 設定要發送的訊息
        }
        data = requests.post(url, headers=headers, data=data)   # 使用 POST 方法
    def continuous_loop(self):
        while True:
            dfs = []
            for code in tqdm(list_):
                try:
                    df = self.backtest_code(code)
                    if len(df) == 0: continue
                    dfs.append(df)
                    
                except Exception as e:
                    print(f'error : {e}')
            heads = pd.concat(dfs).sort_index(ascending=False).head()
            print(pd.concat(dfs).sort_index(ascending=False))
            if list(heads.iloc[:, -1]) != self.heads_code:
                self.heads_code = heads['code']
                self.lineNotify('loop checking', 'c', 'd', self.heads_code)
                time.sleep(60)
            else:
                print(1)
    def lineNotify(self, strat, code, direction, stdized123):
        url = 'https://notify-api.line.me/api/notify'
        line_token = '4YeT18qmmjfGAM40QTmlZqNicCf36sweYK5CAMdEmRS'
        headers = {
            'Authorization': 'Bearer ' + line_token    # 設定權杖
        }
        data = {
            'message':f'{strat}\n{code}\n{direction}\n{datetime.now()}\n{stdized123}'     # 設定要發送的訊息
        }
        data = requests.post(url, headers=headers, data=data)   # 使用 POST 方法
    def backtest_code(self, code, benchmark = 2):
        rolling = 5
        benchmark_index = "QQQ"
        cursor.execute(f"SELECT da, code, cl from public.stock_price where code in ('{code}', '{benchmark_index}')")
        conn.commit()
        res = cursor.fetchall()
        df = pd.DataFrame(res)
        df.columns = ['da','code', 'cl']
        pivoted = df.pivot(columns='code', values='cl', index='da')
        pivoted.ffill()
        ret = pivoted.pct_change(fill_method=None)
        ret['stock_price'] = pivoted[code]
        ret['index_price'] = pivoted[benchmark_index]
        ret['ret_diff'] = ret[code] - ret[benchmark_index]
        # type 2
        ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

        # # check direction
        ret['lag10'] = ret['stock_price'].shift(5)
        ret['change_stock'] = (ret['stock_price'] - ret['lag10'])/ret['lag10']
        ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)
        
        # # check if the direction are the same, if not, tned to not being a mean reversion
        ret['lag10_twii'] = ret['index_price'].shift(5)
        ret['change_twii'] = (ret['index_price'] - ret['lag10_twii'])/ret['lag10_twii']
        ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
        ret['filter_2'] = ret.apply(lambda x: 1 if abs(x['change_stock']) < abs(x['change_twii']) else 0, axis=1)
        ret['code'] = code
        ret = ret[(ret['stdize_ret_diff'] >= benchmark) & (ret['filter_1'] == 1) & (ret['filter_2'] == 1)][['filter_1', 'filter_2', 'stdize_ret_diff', 'direction', 'code']].dropna()
        return ret
def main():
    processor = ContinuousProcessor()
    loop_thread = threading.Thread(target=processor.continuous_loop, daemon=True)
    loop_thread.start()

if __name__ == "__main__":
    main()


  0%|          | 0/98 [00:00<?, ?it/s]

 50%|█████     | 49/98 [00:01<00:01, 26.88it/s]

error : no results to fetch
error : 'CDNS'


 56%|█████▌    | 55/98 [00:02<00:01, 24.90it/s]

error : no results to fetch
error : no results to fetch


...

error : no results to fetch
error : no results to fetch


 95%|█████████▍| 93/98 [00:03<00:00, 25.51it/s]

error : no results to fetch


100%|██████████| 98/98 [00:03<00:00, 25.72it/s]
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_27283/596666839.py", line 37, in continuous_loop
  File "/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/pandas/core/reshape/concat.py", line 382, in concat
    op = _Concatenator(
  File "/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/pandas/core/reshape/concat.py", line 445, in __init__
    

error : no results to fetch
error : Length mismatch: Expected axis has 0 elements, new values have 3 elements
error : Length mismatch: Expected axis has 0 elements, new values have 3 elements


In [74]:
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings("ignore")
def backtest_code(code):
    rolling = 5
    cursor.execute(f"SELECT da, code, cl from public.stock_price where code in ('{code}', 'QQQ')")
    conn.commit()
    res = cursor.fetchall()
    df = pd.DataFrame(res)
    df.columns = ['da','code', 'cl']
    pivoted = df.pivot(columns='code', values='cl', index='da')
    pivoted.ffill()
    ret = pivoted.pct_change()
    ret['ret_diff'] = ret[code] - ret['QQQ']
    ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std() * 100
    ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean() * 100
    ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
    ret[code] = pivoted[code].shift(-1)
    ret['QQQ'] = pivoted['QQQ'].shift(-1)

    ret['index'] = [i for i in range(len(ret))]

    # check raise
    ret[f'{code}_lag60'] = ret[code].shift(-30)
    ret[f'TWII_lag60'] = ret["QQQ"].shift(-30)
    ret['lag10'] = ret[code].shift(-4)

    ret['direction'] = ret.apply(lambda x: -1 if x[code] - x['lag10'] > 0 else 1, axis=1)
    ret['target_60mins'] = abs(np.round((ret[f'{code}_lag60'] - ret[code])/ret[code], 3))

    # ret['signal_defineby_prev_ret'] = 
    # specific print
    ret['da'] = pivoted.index
    ret.set_index('index', inplace=True, drop=False)
    # start = -54 # -268
    start = len(ret) - 200
    end = len(ret)
    bm = 2
    # fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 6), sharex=True)
    # ret.iloc[start:end, [0, 1, 4, 5]].plot(ax=axes, subplots=True, legend=True)
    # for index, row in ret.iloc[start:end, :][ret['stdize_ret_diff'] >= bm].iterrows():  
    #     for ax in axes:
    #         ax.axvline(row['index'], color='red')
    # plt.tight_layout()
    # plt.show()

    # check to long or short
    try:
        if ret[ret['stdize_ret_diff'] >= bm].iloc[-1]['da'].day == 30:
            list_ = [code, ret[ret['stdize_ret_diff'] >= bm].iloc[-1]['direction'], ret[ret['stdize_ret_diff'] >= bm].iloc[-1]['da']]
        return list_
    except:
        return None
    
    
dfs = []
for code in tqdm(list_):
    df = backtest_code(code)
    if df is None: 
        continue
    dfs.append(df)
df12 = pd.DataFrame(dfs, columns = ['code', 'dir', 'da'])
df12.sort_values(by='da', ascending=False)


100%|██████████| 98/98 [00:02<00:00, 47.83it/s]


,code,dir,da
6,ORLY,1,2024-08-30 15:53:00
56,SNPS,1,2024-08-30 15:48:00
65,PDD,1,2024-08-30 15:47:00
26,PANW,1,2024-08-30 15:45:00
64,INTU,-1,2024-08-30 15:40:00
...,...,...,...
24,CDW,1,2024-08-30 10:07:00
30,ADSK,-1,2024-08-30 10:02:00
8,BKNG,-1,2024-08-30 10:01:00
10,ARM,1,2024-08-30 09:47:00


,code,dir,da
30,CPRT,-1,2024-08-30 14:19:00
61,DDOG,1,2024-08-30 14:18:00
28,ASML,-1,2024-08-30 14:13:00
3,PYPL,-1,2024-08-30 14:12:00
16,TTD,1,2024-08-30 14:05:00
...,...,...,...
44,QCOM,-1,2024-08-30 09:46:00
40,DLTR,-1,2024-08-30 09:46:00
29,AAPL,-1,2024-08-30 09:45:00
47,SNPS,-1,2024-08-30 09:42:00


In [72]:
import pandas as pd
import psycopg2
from datetime import datetime
from config import *
import requests
import warnings
import time
import numpy as np
warnings.filterwarnings("ignore")

class SignalGenerator():
    def __init__(self):
        self.list_ = None
        self.init_minute = None
        self.iterator = 0
        self.rolling = 5
        self.benchmark = 2
        self.benchmark_index = "QQQ"
        self.conn = self.get_db_connection()
        self.cursor = self.conn.cursor()
        self.get_code_list(index='us100')
    def get_db_connection(self):
        conn = psycopg2.connect(host=DB_HOST, dbname='us', user=DB_USER, password=DB_PASS)
        return conn
    
    def get_code_list(self, index='us100'):
        self.cursor.execute(f"SELECT distinct code from public.maincode where listed = '{index}';")
        self.conn.commit()
        self.list_ = [i[0] for i in self.cursor.fetchall()]
        return None
    
    def lineNotify(self, code, direction, stdsize123, da):
        headers = {
            'Authorization': 'Bearer ' + LINE_TOKEN
        }
        data = {
            'message':f'''
            ========今天 {self.iterator} 個訊號 ==========
            code: {code}\n 
            direction: {direction}(1 for long, -1 for short)\n 
            current time:{datetime.now()}\n
            signal_time: {da} \n
            signal_value:{stdsize123} \n
            =============================================
            '''             
        }
        data = requests.post(LINE_URL, headers=headers, data=data)
        self.iterator += 1

    def backtest_strat1(self, code):
        '''
        abs(ret_diff - ret_mean)/ret_std
        '''
        self.cursor.execute(f"SELECT da, code, cl from public.stock_price where code in ('{code}', '{self.benchmark_index}') limit 20")
        self.conn.commit()
        df = pd.DataFrame(self.cursor.fetchall())
        df.columns = ['da','code', 'cl']
        pivoted = df.pivot(columns='code', values='cl', index='da')
        pivoted.ffill()
        ret = pivoted.pct_change()
        ret['ret_diff'] = ret[code] - ret[self.benchmark_index]
        ret['ret_diff_std'] = ret['ret_diff'].rolling(self.rolling).std() * 100
        ret['ret_diff_mean'] = ret['ret_diff'].rolling(self.rolling).mean() * 100
        ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])/ret['ret_diff_std'])
        ret[code] = pivoted[code]
        ret[self.benchmark_index] = pivoted[self.benchmark_index]
        ret['da'] = pivoted.index
        last = ret.iloc[-1, :]
        if last[last['stdize_ret_diff'] >= self.benchmark]:
            self.lineNotify(code, '1', np.round(last['stdize_ret_diff']), last['da'])
            return [last['da'], code]
    def backtest_strat2(self, code):
        try:
            self.cursor.execute(f"SELECT da, code, cl from public.stock_price where code in ('{code}', '{self.benchmark_index}') limit 20")
            self.conn.commit()
            df = pd.DataFrame(self.cursor.fetchall())
            df.columns = ['da','code', 'cl']
            pivoted = df.pivot(columns='code', values='cl', index='da')
            pivoted.ffill()
            ret = pivoted.pct_change()
            ret['stock_price'] = pivoted[code]
            ret['index_price'] = pivoted[self.benchmark_index]
            ret['ret_diff'] = ret[code] - ret[self.benchmark_index]
            # type 2
            ret['ret_diff_std'] = ret['ret_diff'].rolling(self.rolling).std()
            ret['ret_diff_mean'] = ret['ret_diff'].rolling(self.rolling).mean()
            ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

            # # check direction
            ret['lag10'] = ret['stock_price'].shift(5)
            ret['change_stock'] = (ret['stock_price'] - ret['lag10'])/ret['lag10']
            ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)
            
            ret['change_twii'] = (ret['index_price'] - ret['lag10_twii'])/ret['lag10_twii']
            ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
            ret['filter_2'] = ret.apply(lambda x: 1 if abs(x['change_stock']) < abs(x['change_twii']) else 1, axis=1)
            last = ret.iloc[-1, :]
            da = last.index[-1]
            if last['stdize_ret_diff'] >= 2 and last['filter_1'] == 1 and last['filter_2'] == 1:
                self.lineNotify(code, last['direction'], last['stdize_ret_diff'], last['filter_1'])
        except Exception as e:
            print(e)
    def signal_generator(self):
        for code in self.list_:
            if code == self.benchmark_index: continue
            self.backtest_strat1(code)
        for code in self.list_:
            if code == self.benchmark_index: continue
            self.backtest_strat2(code)
        
    def flow_controller(self):
        self.cursor.execute("SELECT da from public.stock_price order by da desc limit 1;")
        self.conn.commit()
        return self.cursor.fetchone()[0]
if __name__ == "__main__":
    sg = SignalGenerator()
    while True:
        da = sg.flow_controller()
        if sg.init_minute != da:
            sg.init_minute = da
            sg.signal_generator()
        else:
            time.sleep(60 - datetime.now().second + 3)

KeyError: np.False_